In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image, PointCloud2
from cv_bridge import CvBridge
import cv2
import numpy as np
import pandas as pd
from time import time, sleep
from datetime import datetime

In [ ]:
# Create directories for saving images
import os
left_images_dir = 'left_images'
right_images_dir = 'right_images'
depth_images_dir = 'depth_images'
os.makedirs(left_images_dir, exist_ok=True)
os.makedirs(right_images_dir, exist_ok=True)
os.makedirs(depth_images_dir, exist_ok=True)

In [ ]:
class SimpleDataCollector(Node):
    def __init__(self):
        super().__init__('simple_data_collector')

        self.left_image_sub = self.create_subscription(Image, '/cam_1/zed_node/left/image_rect_color', self.left_image_callback, 10)
        self.right_image_sub = self.create_subscription(Image, '/cam_1/zed_node/right/image_rect_color', self.right_image_callback, 10)
        self.depth_image_sub = self.create_subscription(Image, '/cam_1/zed_node/depth/depth_registered', self.depth_image_callback, 10)
        
        self.last_timestamp = 0
        self.sync_rate = 1/100
        
        self.left_image = None
        self.right_image = None
        self.depth_image = None

        self.left_time = 0
        self.right_time = 0
        self.depth_time = 0

    def synchronization(self):
        if self.left_image is not None and self.right_image is not None and self.depth_image:
            if self.left_time == self.right_time == self.depth_time:
                if self.left_time - self.last_timestamp >= self.sync_rate:

                    print('Current time: ' + str(datetime.fromtimestamp(self.left_time)))
                    print('Interval: ' + str(self.left_time - self.last_timestamp))

                    cv_left = bridge.imgmsg_to_cv2(self.left_image, desired_encoding='bgr8')
                    cv_right = bridge.imgmsg_to_cv2(self.right_image, desired_encoding='bgr8')
                    cv_depth = bridge.imgmsg_to_cv2(self.depth_image, desired_encoding='passthrough')

                    left_filename = f"{left_images_dir}/frame_{self.left_time}.png"
                    right_filename = f"{right_images_dir}/frame_{self.right_time}.png"
                    depth_filename = f"{depth_images_dir}/frame_{self.depth_time}.png"

                    cv2.imwrite(left_filename, cv_left)
                    cv2.imwrite(right_filename, cv_right)
                    cv2.imwrite(depth_filename, cv_depth)

                    self.last_timestamp = self.left_time

    def left_image_callback(self, msg):
        self.left_image = msg
        self.left_time = self.left_image.header.stamp.sec + self.left_image.header.stamp.nanosec / 1e9
        self.synchronization()

    def right_image_callback(self, msg):
        self.right_image = msg
        self.right_time = self.right_image.header.stamp.sec + self.right_image.header.stamp.nanosec / 1e9
        self.synchronization()

    def depth_image_callback(self, msg):
        self.depth_image = msg
        self.depth_time = self.depth_image.header.stamp.sec + self.depth_image.header.stamp.nanosec / 1e9
        self.synchronization()

In [ ]:
bridge = CvBridge()
rclpy.init()
simple_data_collector = SimpleDataCollector()
rclpy.spin(simple_data_collector)